In [28]:
import cv2
from ultralytics import YOLO
import time

def main():
    # YOLO 모델 로드 (weights 파일 경로 지정 필요)
    model = YOLO('/home/kjj73/dev_ws/Project/final_project/best.pt')  # yolov8n.pt, yolov8s.pt 등 선택 가능

    # 웹캠 캡처 시작
    cap = cv2.VideoCapture(0)  # 0은 기본 웹캠을 의미
    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        return

    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 초기 최대 가로 세로 크기

    counter = 15  # 초기 숫자
    multiple = 4.54
    temp = 0
    limit = 55
    
    magnification = (500) / counter

    bot_x, bot_y = 0, 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 가져올 수 없습니다.")
            break

        # YOLO 모델을 사용하여 객체 감지 수행
        results = model(frame)

        # 결과를 OpenCV 이미지로 렌더링
        annotated_frame = results[0].plot()

        # 감지된 객체의 크기를 화면에 표시
        for result in results[0].boxes:  # 각 감지 결과에 대해 반복
            box = result.xyxy[0]  # 경계 상자 좌표 (x1, y1, x2, y2)
            x1, y1, x2, y2 = map(int, box[:4])
            obj_width = x2 - x1
            obj_height = y2 - y1

            # 감지된 객체의 크기 // 
            temp = (obj_width * obj_width) // (magnification * magnification)
            temp2 = limit - (temp // multiple)
            if temp2 <= 10:
                counter = temp2 + 10
            else :
                counter = temp2
                # counter = temp
            
            # 경계 상자 크기 텍스트 추가
            text = f"{obj_width}x{obj_height}"
            cv2.putText(annotated_frame, text, (x1 + 120, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 현재 크기와 숫자를 화면에 표시
        display_text = f"Max Size: {width}x{height}, cm: {counter}"
        cv2.putText(annotated_frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # 화면에 표시
        cv2.imshow('YOLOv8 Real-Time Detection', annotated_frame)

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        time.sleep(0.01)

    # 자원 해제
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



0: 480x640 (no detections), 11.0ms
Speed: 4.1ms preprocess, 11.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 8.7ms
Speed: 3.7ms preprocess, 8.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 8.2ms
Speed: 1.5ms preprocess, 8.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 11.4ms
Speed: 3.4ms preprocess, 11.4ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 14.3ms
Speed: 5.1ms preprocess, 14.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 11.6ms
Speed: 4.3ms preprocess, 11.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 11.1ms
Speed: 4.0ms preprocess, 11.1ms infer

In [ ]:
import cv2
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO('/home/kjj73/dev_ws/Project/final_project/best.pt')  # YOLO 가중치 파일 경로

# 파이 카메라 초기화
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # 해상도 설정
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 탁구공 실제 지름 (단위: mm)
REAL_DIAMETER = 40

# 카메라 초점 거리 (계산된 값)
FOCAL_LENGTH = 4375  # 픽셀 너비 기준으로 계산된 초점 거리

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # YOLO 모델로 객체 감지
    results = model(frame)

    # 프레임에 감지 결과 시각화
    annotated_frame = results[0].plot()

    for result in results[0].boxes:  # 감지된 객체 반복
        box = result.xyxy[0]  # 경계 상자 좌표
        x1, y1, x2, y2 = map(int, box[:4])

        # 중심 좌표 계산
        x_center = int((x1 + x2) / 2)
        y_center = int((y1 + y2) / 2)

        # 탁구공 픽셀 너비 계산
        pixel_width = x2 - x1

        # 거리 계산
        if pixel_width > 0:
            distance = (FOCAL_LENGTH * REAL_DIAMETER) / pixel_width
        else:
            distance = -1  # 유효하지 않은 경우

        # 좌표와 거리 출력
        label = f"Coord: ({x_center}, {y_center}), Dist: {distance:.2f}mm"
        cv2.putText(annotated_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(annotated_frame, (x_center, y_center), 5, (0, 0, 255), -1)

    # 결과 표시
    cv2.imshow("Ping Pong Ball Detection", annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO('/home/kjj73/dev_ws/Project/final_project/best.pt')  # YOLO 가중치 파일 경로

# 카메라 초기화
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 탁구공 실제 지름 (단위: mm)
REAL_DIAMETER = 40

# 초점 거리 (실험적으로 계산된 값)
FOCAL_LENGTH = 687.5  # 초기값

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져올 수 없습니다.")
        break

    # YOLO 모델로 객체 감지
    results = model(frame)

    # 프레임에 감지 결과 시각화
    annotated_frame = results[0].plot()

    for result in results[0].boxes:  # 감지된 객체 반복
        box = result.xyxy[0]  # 경계 상자 좌표
        x1, y1, x2, y2 = map(int, box[:4])

        # 중심 좌표 계산
        x_center = int((x1 + x2) / 2)
        y_center = int((y1 + y2) / 2)

        # 픽셀 너비 계산
        pixel_width = x2 - x1

        # 거리 계산
        if pixel_width > 0:
            distance = (FOCAL_LENGTH * REAL_DIAMETER) / pixel_width
        else:
            distance = -1  # 유효하지 않은 경우

        # 좌표, 픽셀 너비 및 거리 출력
        label1 = f"Coord: ({x_center}, {y_center})"
        label2 = f"Width: {pixel_width}px"
        label3 = f"Dist: {distance:.2f}mm"
        cv2.circle(annotated_frame, (x_center, y_center), 5, (0, 0, 255), -1)  # 중심점 표시

        cv2.putText(annotated_frame, label3, (x1+5, y1 + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        cv2.putText(annotated_frame, label1, (x1+5, y1 + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        cv2.putText(annotated_frame, label2, (x1+5, y1 + 55), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 경계 상자 표시

    # 결과 표시
    cv2.imshow("Ping Pong Ball Detection", annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 229.5ms
Speed: 6.7ms preprocess, 229.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 237.1ms
Speed: 3.9ms preprocess, 237.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 241.8ms
Speed: 4.1ms preprocess, 241.8ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 oball, 193.4ms
Speed: 17.4ms preprocess, 193.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 282.1ms
Speed: 3.2ms preprocess, 282.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 oball, 218.6ms
Speed: 8.1ms preprocess, 218.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 198.5ms
Speed: 13.1ms preprocess, 198.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 160.7ms
Speed: 9.5ms preprocess, 160.7ms